# Episode 1: The Phantom [Table] Menace

This notebook is meant for in-house demonstration of candidate extraction and featurization of tables. It assumes an input file in XHTML format, a strict form of HTML that coincides with XML structure, allowing for easy display (HTML) and safe tree traversal (XML).

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Candidate Extraction

First, import the 'HTMLParser' class to read HTML tables

In [40]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/diseases.xhtml')

The "TableParser" class divides the html doc into cells, adding a 'cell_id' attribute to each cell for future traversal, and creating "Cell" objects that have attributes such as row number, column number, html tag, html attributes, and any tags/attributes on a cells ancestors in the table.

In [41]:
from snorkel.parser import TableParser
table_parser = TableParser()

As usual, pass these to a Corpus object for digestion.

In [42]:
from snorkel.parser import Corpus
%time corpus = Corpus(html_parser, table_parser)

Parsing documents...
0
Parsing contexts...
CPU times: user 85.6 ms, sys: 5.34 ms, total: 91 ms
Wall time: 134 ms


Load the good 'ole disease dictionary for recognizing disease names.

In [8]:
from load_dictionaries import load_disease_dictionary

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

Loaded 507899 disease phrases!


Here we use a new CandidateSpace object, CellNgrams. It inherits from Ngrams, and ensures that the Table context object is broken up into cells before being passed into the usual routine for pulling out Ngrams.

In [43]:
from snorkel.candidates import CellNgrams
from snorkel.matchers import DictionaryMatch

# Define a candidate space
cell_ngrams = CellNgrams(n_max=3)

# Define a matcher
disease_matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Passing the CandidateSpace, Matcher, and Context objects to a Candidates object, extraction is performed, and we see that a number of disease CellNgrams are returned.

In [52]:
from snorkel.candidates import Candidates
%time candidates = Candidates(cell_ngrams, disease_matcher, corpus.get_contexts())
for candy in candidates.get_candidates(): print candy

Extracting candidates...
CPU times: user 2.73 ms, sys: 734 µs, total: 3.46 ms
Wall time: 2.94 ms
<CellNgram("Hypochondria", id=0-1-22-0:0-11, chars=[0,11], (row,col)=(3,1), tag=th)
<CellNgram("Scurvy", id=0-0-22-0:0-5, chars=[0,5], (row,col)=(3,1), tag=th)
<CellNgram("Location", id=0-0-3-0:0-7, chars=[0,7], (row,col)=(0,3), tag=th)
<CellNgram("Polio", id=0-0-8-0:0-4, chars=[0,4], (row,col)=(1,1), tag=th)
<CellNgram("Yellow Fever", id=0-1-15-0:0-11, chars=[0,11], (row,col)=(2,1), tag=th)
<CellNgram("Fever", id=0-1-15-0:7-11, chars=[7,11], (row,col)=(2,1), tag=th)
<CellNgram("Disease", id=0-0-1-0:0-6, chars=[0,6], (row,col)=(0,1), tag=th)
<CellNgram("Problem", id=0-1-1-0:0-6, chars=[0,6], (row,col)=(0,1), tag=th)
<CellNgram("Arthritis", id=0-1-8-0:0-8, chars=[0,8], (row,col)=(1,1), tag=th)
<CellNgram("Chicken Pox", id=0-0-15-0:0-10, chars=[0,10], (row,col)=(2,1), tag=th)


### Feature Generation

We can then generate features on our set of candidates, including *new and improved* table features!

In [54]:
from snorkel.features import CellNgramFeaturizer
featurizer = CellNgramFeaturizer(candidates, corpus)

Extracting features...
Extracted 69 features for each of 10 mentions


In [55]:
featurizer.get_features_by_id('0-0-8-0:0-4')

['Ngram_features_to_come',
 'TABLE_ROW_NUM_1',
 'TABLE_COL_NUM_1',
 'TABLE_HTML_TAG_th',
 'TABLE_HTML_ATTR_c=e=l=l=_=i=d',
 'TABLE_HTML_ANC_TAG_tr',
 'TABLE_HTML_ANC_TAG_tbody',
 'TABLE_HTML_ANC_TAG_table',
 'TABLE_HTML_ANC_TAG_body',
 'TABLE_HTML_ANC_ATTR_font=blue',
 u'TABLE_ROW_NGRAM_Polio',
 u'TABLE_ROW_NGRAM_1914',
 u'TABLE_ROW_NGRAM_New',
 u'TABLE_ROW_NGRAM_New York',
 u'TABLE_ROW_NGRAM_York',
 u'TABLE_COL_NGRAM_Polio',
 u'TABLE_COL_NGRAM_Disease',
 u'TABLE_COL_NGRAM_Scurvy',
 u'TABLE_COL_NGRAM_Chicken',
 u'TABLE_COL_NGRAM_Chicken Pox',
 u'TABLE_COL_NGRAM_Pox']

Ta-da! Next up: feeding this features into the learning machine.